# Import dependencies

In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecTransposeImage, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

# Test enviroment

In [9]:
enviroment_name = 'Blackjack-v1'
env = gym.make(enviroment_name, render_mode='human')

In [3]:
env.reset()

((13, 3, 1), {})

In [4]:
# 0: stick, 1: hit
env.action_space

Discrete(2)

In [5]:
# The player’s current sum, the value of the dealer’s one showing card (1-10 where 1 is ace), and whether the player holds a usable ace (0 or 1)
env.observation_space

Tuple(Discrete(32), Discrete(11), Discrete(2))

In [18]:
env.observation_space.sample()

(17, 5, 1)

In [6]:
env.render()

In [7]:
env.close()

In [10]:
episodes = 5
for episode in range(1, episodes+1):
  obs = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    score += reward

  print('Episode: {} Score {}'.format(episode, score))
#env.close()

Episode: 1 Score -1.0
Episode: 2 Score -1.0
Episode: 3 Score 1.0
Episode: 4 Score -1.0
Episode: 5 Score -1.0


# Train model

In [ ]:
env = gym.make(enviroment_name, render_mode='human')
env = gym.wrappers.FlattenObservation(env)
env = DummyVecEnv([lambda: env])

In [25]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [26]:
model.learn(total_timesteps=10000)

Logging to Training\Logs\PPO_7
-----------------------------
| time/              |      |
|    fps             | 2    |
|    iterations      | 1    |
|    time_elapsed    | 888  |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2           |
|    iterations           | 2           |
|    time_elapsed         | 1800        |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015928742 |
|    clip_fraction        | 0.289       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.677      |
|    explained_variance   | -0.169      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.319       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0456     |
|    value_loss           | 0.747       |
-----------------------------------------
---

# Save model

In [27]:
Lunar_path = os.path.join('Training', 'Saved models', 'PPO_Blackjack_Model')
model.save(Lunar_path)

In [28]:
del model

In [29]:
model = PPO.load(Lunar_path, env)

# Evaluate and test

In [30]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=True)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

c:\Users\william.mendes\Anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:0.00 +/- 1.00


In [ ]:
env.close()

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
  obs = env.reset()[0]
  done = False
  score = 0

  while not done:
    env.render()
    action = model.predict(obs)[0]
    obs, reward, done, truncated, info = env.step(action)
    score += reward

  print('Episode: {} Score {}'.format(episode, score))
#env.close()